In [1]:
from utils import *
import os
import pandas as pd
import sklearn
from sklearn import *
import pickle

In [2]:
path_folder_quora = "./Datasets/QuoraQuestionPairs/"

In [3]:
train_df = pd.read_csv(os.path.join(path_folder_quora, "quora_train_data.csv"))
A_df, te_df = sklearn.model_selection.train_test_split(train_df,test_size=0.05,random_state=123)
tr_df, va_df = sklearn.model_selection.train_test_split(A_df,test_size=0.05,random_state=123)
print('tr_df.shape=',tr_df.shape)
print('va_df.shape=',va_df.shape)
print('te_df.shape=',te_df.shape)

tr_df.shape= (291897, 6)
va_df.shape= (15363, 6)
te_df.shape= (16172, 6)


In [4]:
q1_train =  cast_list_as_strings(list(tr_df["question1"]))
q2_train =  cast_list_as_strings(list(tr_df["question2"]))
q1_validation =  cast_list_as_strings(list(va_df["question1"]))
q2_validation =  cast_list_as_strings(list(va_df["question2"]))
q1_test = cast_list_as_strings(list(te_df["question1"]))
q2_test = cast_list_as_strings(list(te_df["question2"]))

In [5]:
count_vectorizer = pickle.load(open('model_artifacts/count_vectorizer.pkl', 'rb'))
logistic = pickle.load(open('model_artifacts/logistic.pkl', 'rb'))

In [6]:
X_tr_q1q2 = get_features_from_df(tr_df, count_vectorizer)
X_va_q1q2 = get_features_from_df(va_df, count_vectorizer)
X_te_q1q2 = get_features_from_df(te_df, count_vectorizer)

X_tr_q1q2.shape, tr_df.shape, X_va_q1q2.shape, va_df.shape, X_te_q1q2.shape, te_df.shape

((291897, 149650),
 (291897, 6),
 (15363, 149650),
 (15363, 6),
 (16172, 149650),
 (16172, 6))

In [7]:
y_train = tr_df["is_duplicate"].values
y_validation = va_df["is_duplicate"].values
y_test = te_df["is_duplicate"].values

auc_tr = metrics.roc_auc_score(y_train, logistic.predict(X_tr_q1q2))
auc_va = metrics.roc_auc_score(y_validation, logistic.predict(X_va_q1q2))
auc_te = metrics.roc_auc_score(y_test, logistic.predict(X_te_q1q2))

print('Training AUC Score: ' + str(auc_tr))
print('Validation AUC Score: ' + str(auc_va))
print('Test AUC Score: ' + str(auc_te))

Training AUC Score: 0.7874902327238901
Validation AUC Score: 0.7202777566666142
Test AUC Score: 0.7293595330779445


In [8]:
print(metrics.classification_report(y_train, logistic.predict(X_tr_q1q2)))

              precision    recall  f1-score   support

           0       0.83      0.89      0.86    184298
           1       0.78      0.69      0.73    107599

    accuracy                           0.81    291897
   macro avg       0.81      0.79      0.79    291897
weighted avg       0.81      0.81      0.81    291897



In [9]:
print(metrics.classification_report(y_validation, logistic.predict(X_va_q1q2)))

              precision    recall  f1-score   support

           0       0.78      0.83      0.81      9696
           1       0.68      0.61      0.64      5667

    accuracy                           0.75     15363
   macro avg       0.73      0.72      0.72     15363
weighted avg       0.75      0.75      0.75     15363



In [10]:
print(metrics.classification_report(y_test, logistic.predict(X_te_q1q2)))

              precision    recall  f1-score   support

           0       0.79      0.84      0.81     10165
           1       0.70      0.62      0.65      6007

    accuracy                           0.76     16172
   macro avg       0.74      0.73      0.73     16172
weighted avg       0.75      0.76      0.75     16172

